<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/225_Cross_Sell_Upsell_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routine analysis utilities for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""Routine analysis utilities for Tier 2 Cross-Sell & Upsell Orchestrator"""

from typing import Dict, List, Any, Set
from datetime import datetime, timedelta


def analyze_customer_routine(
    customer: Dict[str, Any],
    product_lookup: Dict[str, Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Analyze customer's current routine: extract products, categories, and routine steps.

    Args:
        customer: Customer data dict with products_owned and categories
        product_lookup: Dictionary mapping product_id to product dict

    Returns:
        Dictionary with:
        - customer_products: List of product_ids customer owns
        - customer_categories: List of categories customer has products in
        - customer_routine_steps: List of routine steps customer has (1-5)
    """
    customer_products = []
    customer_categories_set: Set[str] = set()
    customer_routine_steps_set: Set[int] = set()

    # Extract from products_owned
    for product_owned in customer.get("products_owned", []):
        product_id = product_owned.get("product_id")
        if product_id:
            customer_products.append(product_id)

            # Look up product details
            product = product_lookup.get(product_id)
            if product:
                # Add category
                category = product.get("category")
                if category:
                    customer_categories_set.add(category)

                # Add routine step
                routine_step = product.get("routine_step")
                if routine_step is not None:
                    customer_routine_steps_set.add(routine_step)

    # Also check customer's categories field (may have more categories)
    customer_categories_list = customer.get("categories", [])
    if customer_categories_list:
        customer_categories_set.update(customer_categories_list)

    return {
        "customer_products": customer_products,
        "customer_categories": sorted(list(customer_categories_set)),
        "customer_routine_steps": sorted(list(customer_routine_steps_set))
    }


def identify_routine_gaps(
    customer_categories: List[str],
    customer_routine_steps: List[int],
    essential_categories: List[str],
    routine_steps_total: int = 5
) -> Dict[str, Any]:
    """
    Identify routine gaps: missing essential categories and missing routine steps.

    Args:
        customer_categories: List of categories customer has
        customer_routine_steps: List of routine steps customer has (1-5)
        essential_categories: List of essential categories (e.g., ["cleanser", "moisturizer", "spf"])
        routine_steps_total: Total number of routine steps (default: 5)

    Returns:
        Dictionary with:
        - routine_gaps: List of missing essential categories
        - routine_step_gaps: List of missing routine steps (1-5)
    """
    # Find missing essential categories
    customer_categories_set = set(customer_categories)
    essential_categories_set = set(essential_categories)
    routine_gaps = sorted(list(essential_categories_set - customer_categories_set))

    # Find missing routine steps
    customer_routine_steps_set = set(customer_routine_steps)
    all_routine_steps = set(range(1, routine_steps_total + 1))
    routine_step_gaps = sorted(list(all_routine_steps - customer_routine_steps_set))

    return {
        "routine_gaps": routine_gaps,
        "routine_step_gaps": routine_step_gaps
    }


def check_replenishment_needs(
    customer: Dict[str, Any],
    product_lookup: Dict[str, Dict[str, Any]],
    replenishment_warning_days: int = 7
) -> List[Dict[str, Any]]:
    """
    Check which products need replenishment based on purchase date and cycle.

    Args:
        customer: Customer data dict with products_owned
        product_lookup: Dictionary mapping product_id to product dict
        replenishment_warning_days: Days before due date to flag as "approaching" (default: 7)

    Returns:
        List of replenishment needs, each with:
        - product_id: Product ID
        - days_since_purchase: Days since last purchase
        - replenishment_cycle_days: Expected cycle in days
        - replenishment_due: Boolean (True if days_since_purchase >= cycle)
        - approaching_replenishment: Boolean (True if within warning days)
    """
    replenishment_needs = []
    today = datetime.now().date()

    for product_owned in customer.get("products_owned", []):
        product_id = product_owned.get("product_id")
        purchase_date_str = product_owned.get("purchase_date")

        if not product_id or not purchase_date_str:
            continue

        # Look up product to get replenishment cycle
        product = product_lookup.get(product_id)
        if not product:
            continue

        replenishment_cycle_days = product.get("replenishment_cycle_days")
        if replenishment_cycle_days is None:
            continue

        # Parse purchase date
        try:
            purchase_date = datetime.strptime(purchase_date_str, "%Y-%m-%d").date()
        except (ValueError, TypeError):
            continue

        # Calculate days since purchase
        days_since_purchase = (today - purchase_date).days

        # Check if replenishment is due
        replenishment_due = days_since_purchase >= replenishment_cycle_days

        # Check if approaching replenishment (within warning days)
        days_until_due = replenishment_cycle_days - days_since_purchase
        approaching_replenishment = (
            not replenishment_due and
            days_until_due <= replenishment_warning_days and
            days_until_due > 0
        )

        replenishment_needs.append({
            "product_id": product_id,
            "days_since_purchase": days_since_purchase,
            "replenishment_cycle_days": replenishment_cycle_days,
            "replenishment_due": replenishment_due,
            "approaching_replenishment": approaching_replenishment,
            "purchase_date": purchase_date_str
        })

    return replenishment_needs



# Tests for routine analysis utilities

In [ ]:
"""Tests for routine analysis utilities"""

from agents.tier2_crosssell_upsell.utilities.routine_analysis import (
    analyze_customer_routine,
    identify_routine_gaps,
    check_replenishment_needs
)
from agents.tier2_crosssell_upsell.utilities.data_loading import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)


def test_analyze_customer_routine():
    """Test analyze_customer_routine extracts routine correctly"""
    print("\n=== Testing analyze_customer_routine ===")

    # Load real data
    customer = load_customer_data("C001")
    assert customer is not None, "Should load customer C001"

    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Analyze routine
    result = analyze_customer_routine(customer, product_lookup)

    # Verify structure
    assert "customer_products" in result, "Should return customer_products"
    assert "customer_categories" in result, "Should return customer_categories"
    assert "customer_routine_steps" in result, "Should return customer_routine_steps"

    # Verify customer_products
    assert isinstance(result["customer_products"], list), "customer_products should be a list"
    assert len(result["customer_products"]) > 0, "Should have at least one product"

    # Verify customer_categories
    assert isinstance(result["customer_categories"], list), "customer_categories should be a list"
    assert len(result["customer_categories"]) > 0, "Should have at least one category"

    # Verify customer_routine_steps
    assert isinstance(result["customer_routine_steps"], list), "customer_routine_steps should be a list"
    assert all(1 <= step <= 5 for step in result["customer_routine_steps"]), "Steps should be 1-5"

    print("✅ analyze_customer_routine test passed!")
    print(f"   Products: {len(result['customer_products'])}")
    print(f"   Categories: {result['customer_categories']}")
    print(f"   Routine steps: {result['customer_routine_steps']}")

    return result


def test_identify_routine_gaps():
    """Test identify_routine_gaps finds missing categories and steps"""
    print("\n=== Testing identify_routine_gaps ===")

    # Test with customer who has all essentials (C001)
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    routine_analysis = analyze_customer_routine(customer, product_lookup)

    essential_categories = ["cleanser", "moisturizer", "spf"]
    gaps = identify_routine_gaps(
        routine_analysis["customer_categories"],
        routine_analysis["customer_routine_steps"],
        essential_categories,
        routine_steps_total=5
    )

    # Verify structure
    assert "routine_gaps" in gaps, "Should return routine_gaps"
    assert "routine_step_gaps" in gaps, "Should return routine_step_gaps"

    assert isinstance(gaps["routine_gaps"], list), "routine_gaps should be a list"
    assert isinstance(gaps["routine_step_gaps"], list), "routine_step_gaps should be a list"

    print("✅ identify_routine_gaps test passed!")
    print(f"   Routine gaps (missing categories): {gaps['routine_gaps']}")
    print(f"   Routine step gaps: {gaps['routine_step_gaps']}")

    # Test with customer missing essentials (C002)
    customer2 = load_customer_data("C002")
    if customer2:
        routine_analysis2 = analyze_customer_routine(customer2, product_lookup)
        gaps2 = identify_routine_gaps(
            routine_analysis2["customer_categories"],
            routine_analysis2["customer_routine_steps"],
            essential_categories,
            routine_steps_total=5
        )
        print(f"   C002 gaps: {gaps2['routine_gaps']}, step gaps: {gaps2['routine_step_gaps']}")

    return gaps


def test_check_replenishment_needs():
    """Test check_replenishment_needs calculates replenishment correctly"""
    print("\n=== Testing check_replenishment_needs ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Check replenishment needs
    replenishment_needs = check_replenishment_needs(customer, product_lookup)

    # Verify structure
    assert isinstance(replenishment_needs, list), "Should return a list"

    # Verify each need has required fields
    for need in replenishment_needs:
        assert "product_id" in need, "Each need should have product_id"
        assert "days_since_purchase" in need, "Each need should have days_since_purchase"
        assert "replenishment_cycle_days" in need, "Each need should have replenishment_cycle_days"
        assert "replenishment_due" in need, "Each need should have replenishment_due"
        assert "approaching_replenishment" in need, "Each need should have approaching_replenishment"

        assert isinstance(need["days_since_purchase"], int), "days_since_purchase should be int"
        assert isinstance(need["replenishment_due"], bool), "replenishment_due should be bool"
        assert isinstance(need["approaching_replenishment"], bool), "approaching_replenishment should be bool"

    print("✅ check_replenishment_needs test passed!")
    print(f"   Replenishment needs found: {len(replenishment_needs)}")

    # Show some examples
    for need in replenishment_needs[:3]:
        print(f"   Product {need['product_id']}: {need['days_since_purchase']} days since purchase, "
              f"cycle={need['replenishment_cycle_days']}, due={need['replenishment_due']}")

    return replenishment_needs


def test_routine_analysis_integration():
    """Test all routine analysis utilities together"""
    print("\n=== Testing routine analysis integration ===")

    # Load data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Step 1: Analyze routine
    routine_analysis = analyze_customer_routine(customer, product_lookup)

    # Step 2: Identify gaps
    essential_categories = ["cleanser", "moisturizer", "spf"]
    gaps = identify_routine_gaps(
        routine_analysis["customer_categories"],
        routine_analysis["customer_routine_steps"],
        essential_categories,
        routine_steps_total=5
    )

    # Step 3: Check replenishment
    replenishment_needs = check_replenishment_needs(customer, product_lookup)

    print("✅ Integration test passed!")
    print(f"   Customer: {customer['name']} ({customer['customer_id']})")
    print(f"   Products: {len(routine_analysis['customer_products'])}")
    print(f"   Categories: {routine_analysis['customer_categories']}")
    print(f"   Routine steps: {routine_analysis['customer_routine_steps']}")
    print(f"   Missing categories: {gaps['routine_gaps']}")
    print(f"   Missing steps: {gaps['routine_step_gaps']}")
    print(f"   Replenishment needs: {len(replenishment_needs)}")

    return {
        "routine_analysis": routine_analysis,
        "gaps": gaps,
        "replenishment_needs": replenishment_needs
    }


if __name__ == "__main__":
    """Run all tests"""
    print("=" * 60)
    print("Testing Tier 2 Routine Analysis Utilities")
    print("=" * 60)

    # Test individual utilities
    test_analyze_customer_routine()
    test_identify_routine_gaps()
    test_check_replenishment_needs()

    # Integration test
    test_routine_analysis_integration()

    print("\n" + "=" * 60)
    print("✅ All routine analysis utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_routine_analysis
============================================================
Testing Tier 2 Routine Analysis Utilities
============================================================

=== Testing analyze_customer_routine ===
✅ analyze_customer_routine test passed!
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]

=== Testing identify_routine_gaps ===
✅ identify_routine_gaps test passed!
   Routine gaps (missing categories): []
   Routine step gaps: [2]
   C002 gaps: ['moisturizer', 'spf'], step gaps: [4, 5]

=== Testing check_replenishment_needs ===
✅ check_replenishment_needs test passed!
   Replenishment needs found: 6
   Product P102: 23 days since purchase, cycle=30, due=False
   Product P105: 21 days since purchase, cycle=30, due=False
   Product P108: 18 days since purchase, cycle=30, due=False

=== Testing routine analysis integration ===
✅ Integration test passed!
   Customer: Sarah Lee (C001)
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

============================================================
✅ All routine analysis utility tests passed!
============================================================


#Opportunity detection utilities for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""Opportunity detection utilities for Tier 2 Cross-Sell & Upsell Orchestrator"""

from typing import Dict, List, Any, Set


def find_cross_sell_opportunities(
    customer_routine_steps: List[int],
    customer_products: List[str],
    product_catalog: List[Dict[str, Any]],
    product_lookup: Dict[str, Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Find cross-sell opportunities based on routine step adjacency.

    Looks for products in missing routine steps, prioritizing adjacent steps.
    For example, if customer has step 1, suggest step 2.

    Args:
        customer_routine_steps: List of routine steps customer has (1-5)
        customer_products: List of product_ids customer owns
        product_catalog: List of all Tier 2 products
        product_lookup: Dictionary mapping product_id to product dict

    Returns:
        List of cross-sell opportunities, each with:
        - product_id, product_name, category, routine_step, price, margin
        - recommendation_type: "routine_step_gap"
        - rationale: Explanation of why this is recommended
    """
    opportunities = []
    customer_products_set = set(customer_products)
    customer_steps_set = set(customer_routine_steps)

    # Find missing routine steps
    all_steps = set(range(1, 6))  # Steps 1-5
    missing_steps = sorted(list(all_steps - customer_steps_set))

    # For each missing step, find products in that step
    for step in missing_steps:
        for product in product_catalog:
            product_id = product.get("product_id")

            # Skip if customer already owns this product
            if product_id in customer_products_set:
                continue

            # Check if product is in this routine step
            if product.get("routine_step") == step:
                # Prioritize adjacent steps (e.g., if customer has step 1, step 2 is more relevant)
                is_adjacent = False
                if step > 1 and (step - 1) in customer_steps_set:
                    is_adjacent = True
                elif step < 5 and (step + 1) in customer_steps_set:
                    is_adjacent = True

                opportunity = {
                    "product_id": product_id,
                    "product_name": product.get("name", ""),
                    "category": product.get("category", ""),
                    "routine_step": step,
                    "price": product.get("price", 0.0),
                    "margin": product.get("margin", "medium"),
                    "active_ingredients": product.get("active_ingredients", []),
                    "target_concerns": product.get("target_concerns", []),
                    "recommendation_type": "routine_step_gap",
                    "rationale": f"Customer missing routine step {step}. "
                               f"{'Adjacent to existing steps' if is_adjacent else 'Completes routine sequence'}."
                }
                opportunities.append(opportunity)

    return opportunities


def find_ingredient_based_opportunities(
    customer: Dict[str, Any],
    customer_products: List[str],
    product_catalog: List[Dict[str, Any]],
    product_lookup: Dict[str, Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Find opportunities based on matching customer skin_concerns to product target_concerns.

    Args:
        customer: Customer data dict with skin_concerns
        customer_products: List of product_ids customer owns
        product_catalog: List of all Tier 2 products
        product_lookup: Dictionary mapping product_id to product dict

    Returns:
        List of ingredient-based opportunities, each with:
        - product_id, product_name, category, routine_step, price, margin
        - recommendation_type: "ingredient_match"
        - rationale: Explanation of ingredient/concern match
        - match_score: 1.0 (exact), 0.5 (partial), 0.0 (no match)
    """
    opportunities = []
    customer_products_set = set(customer_products)
    customer_concerns = set(customer.get("skin_concerns", []))

    if not customer_concerns:
        return opportunities  # No concerns to match

    for product in product_catalog:
        product_id = product.get("product_id")

        # Skip if customer already owns this product
        if product_id in customer_products_set:
            continue

        # Get product's target concerns
        product_concerns = set(product.get("target_concerns", []))

        # Calculate match score
        if customer_concerns & product_concerns:  # Intersection (exact matches)
            match_score = 1.0
            matched_concerns = list(customer_concerns & product_concerns)
            match_type = "exact"
        else:
            # No exact match, but still possible if routine gap
            match_score = 0.0
            matched_concerns = []
            match_type = "none"

        # Only include if there's a match (or we want to include all for routine gaps)
        # For now, only include if there's a match
        if match_score > 0:
            opportunity = {
                "product_id": product_id,
                "product_name": product.get("name", ""),
                "category": product.get("category", ""),
                "routine_step": product.get("routine_step", 0),
                "price": product.get("price", 0.0),
                "margin": product.get("margin", "medium"),
                "active_ingredients": product.get("active_ingredients", []),
                "target_concerns": product.get("target_concerns", []),
                "recommendation_type": "ingredient_match",
                "rationale": f"Product targets {', '.join(matched_concerns)} concerns which match customer's skin concerns.",
                "match_score": match_score,
                "matched_concerns": matched_concerns
            }
            opportunities.append(opportunity)

    return opportunities


def find_upgrade_opportunities(
    customer: Dict[str, Any],
    customer_products: List[str],
    product_catalog: List[Dict[str, Any]],
    product_lookup: Dict[str, Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Find upgrade opportunities from Tier 1 to Tier 2 products.

    Checks if customer has Tier 1 products that can be upgraded to Tier 2.
    Only for customers with tier_preference >= 2.

    Args:
        customer: Customer data dict with tier_preference
        customer_products: List of product_ids customer owns (Tier 2)
        product_catalog: List of all Tier 2 products
        product_lookup: Dictionary mapping product_id to product dict

    Returns:
        List of upgrade opportunities, each with:
        - product_id, product_name, category, routine_step, price, margin
        - recommendation_type: "upgrade"
        - rationale: Explanation of upgrade path
        - upgrades_from: List of Tier 1 product IDs this upgrades from
    """
    opportunities = []
    customer_products_set = set(customer_products)
    tier_preference = customer.get("tier_preference", 1)

    # Only suggest upgrades for customers with tier_preference >= 2
    if tier_preference < 2:
        return opportunities

    # Check purchase_history for Tier 1 products (P001-P020)
    # Note: Tier 2 customers may not have Tier 1 products in their data
    # This is a simplified check - in reality, we'd need access to Tier 1 product data
    purchase_history = customer.get("purchase_history", [])
    tier1_products = set()

    for purchase in purchase_history:
        product_id = purchase.get("product_id", "")
        # Tier 1 products are P001-P020 (or P010-P020 based on data structure)
        # For now, we'll check if product_id starts with "P0" and is not in Tier 2 range
        if product_id.startswith("P0") and not product_id.startswith("P1"):
            tier1_products.add(product_id)

    # If no Tier 1 products found, return empty (customer is already on Tier 2)
    if not tier1_products:
        return opportunities

    # Find Tier 2 products that upgrade from customer's Tier 1 products
    for product in product_catalog:
        product_id = product.get("product_id")

        # Skip if customer already owns this Tier 2 product
        if product_id in customer_products_set:
            continue

        # Check if this product upgrades from any of customer's Tier 1 products
        upgrades_from = product.get("upgrades_from", [])
        matching_tier1 = list(set(upgrades_from) & tier1_products)

        if matching_tier1:
            opportunity = {
                "product_id": product_id,
                "product_name": product.get("name", ""),
                "category": product.get("category", ""),
                "routine_step": product.get("routine_step", 0),
                "price": product.get("price", 0.0),
                "margin": product.get("margin", "medium"),
                "active_ingredients": product.get("active_ingredients", []),
                "target_concerns": product.get("target_concerns", []),
                "recommendation_type": "upgrade",
                "rationale": f"Upgrade from Tier 1 product(s) {', '.join(matching_tier1)} to Tier 2 with enhanced ingredients.",
                "upgrades_from": matching_tier1
            }
            opportunities.append(opportunity)

    return opportunities


def find_replenishment_opportunities(
    replenishment_needs: List[Dict[str, Any]],
    product_lookup: Dict[str, Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Convert replenishment needs into opportunities.

    Args:
        replenishment_needs: List of replenishment needs from check_replenishment_needs
        product_lookup: Dictionary mapping product_id to product dict

    Returns:
        List of replenishment opportunities, each with:
        - product_id, product_name, category, routine_step, price, margin
        - recommendation_type: "replenishment"
        - rationale: Explanation of replenishment need
        - days_since_purchase, replenishment_due, approaching_replenishment
    """
    opportunities = []

    for need in replenishment_needs:
        product_id = need.get("product_id")
        product = product_lookup.get(product_id)

        if not product:
            continue

        # Only create opportunity if replenishment is due or approaching
        if need.get("replenishment_due") or need.get("approaching_replenishment"):
            opportunity = {
                "product_id": product_id,
                "product_name": product.get("name", ""),
                "category": product.get("category", ""),
                "routine_step": product.get("routine_step", 0),
                "price": product.get("price", 0.0),
                "margin": product.get("margin", "medium"),
                "active_ingredients": product.get("active_ingredients", []),
                "target_concerns": product.get("target_concerns", []),
                "recommendation_type": "replenishment",
                "rationale": f"Product due for replenishment. "
                           f"{'Replenishment overdue' if need.get('replenishment_due') else 'Replenishment approaching'} "
                           f"({need.get('days_since_purchase')} days since purchase, "
                           f"cycle: {need.get('replenishment_cycle_days')} days).",
                "days_since_purchase": need.get("days_since_purchase", 0),
                "replenishment_due": need.get("replenishment_due", False),
                "approaching_replenishment": need.get("approaching_replenishment", False)
            }
            opportunities.append(opportunity)

    return opportunities



# Tests for opportunity detection utilities

In [ ]:
"""Tests for opportunity detection utilities"""

from agents.tier2_crosssell_upsell.utilities.opportunity_detection import (
    find_cross_sell_opportunities,
    find_ingredient_based_opportunities,
    find_upgrade_opportunities,
    find_replenishment_opportunities
)
from agents.tier2_crosssell_upsell.utilities.data_loading import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)
from agents.tier2_crosssell_upsell.utilities.routine_analysis import (
    analyze_customer_routine
)


def test_find_cross_sell_opportunities():
    """Test find_cross_sell_opportunities finds missing routine steps"""
    print("\n=== Testing find_cross_sell_opportunities ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Analyze routine
    routine_analysis = analyze_customer_routine(customer, product_lookup)

    # Find cross-sell opportunities
    opportunities = find_cross_sell_opportunities(
        routine_analysis["customer_routine_steps"],
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    # Verify structure
    assert isinstance(opportunities, list), "Should return a list"

    # Verify each opportunity has required fields
    for opp in opportunities:
        assert "product_id" in opp, "Each opportunity should have product_id"
        assert "product_name" in opp, "Each opportunity should have product_name"
        assert "routine_step" in opp, "Each opportunity should have routine_step"
        assert "recommendation_type" in opp, "Each opportunity should have recommendation_type"
        assert opp["recommendation_type"] == "routine_step_gap", "Should be routine_step_gap type"
        assert "rationale" in opp, "Each opportunity should have rationale"

    print("✅ find_cross_sell_opportunities test passed!")
    print(f"   Opportunities found: {len(opportunities)}")

    # C001 has steps [1, 3, 4, 5], missing step 2
    if opportunities:
        print(f"   Sample: {opportunities[0]['product_name']} (step {opportunities[0]['routine_step']})")
        print(f"   Rationale: {opportunities[0]['rationale']}")

    return opportunities


def test_find_ingredient_based_opportunities():
    """Test find_ingredient_based_opportunities matches concerns"""
    print("\n=== Testing find_ingredient_based_opportunities ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Analyze routine
    routine_analysis = analyze_customer_routine(customer, product_lookup)

    # C001 has concerns: ["dryness", "sensitivity"]
    # Find ingredient-based opportunities
    opportunities = find_ingredient_based_opportunities(
        customer,
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    # Verify structure
    assert isinstance(opportunities, list), "Should return a list"

    # Verify each opportunity has required fields
    for opp in opportunities:
        assert "product_id" in opp, "Each opportunity should have product_id"
        assert "recommendation_type" in opp, "Each opportunity should have recommendation_type"
        assert opp["recommendation_type"] == "ingredient_match", "Should be ingredient_match type"
        assert "match_score" in opp, "Each opportunity should have match_score"
        assert "matched_concerns" in opp, "Each opportunity should have matched_concerns"
        assert opp["match_score"] > 0, "Match score should be > 0 for ingredient matches"

    print("✅ find_ingredient_based_opportunities test passed!")
    print(f"   Opportunities found: {len(opportunities)}")

    if opportunities:
        print(f"   Sample: {opportunities[0]['product_name']}")
        print(f"   Matched concerns: {opportunities[0]['matched_concerns']}")
        print(f"   Match score: {opportunities[0]['match_score']}")

    return opportunities


def test_find_upgrade_opportunities():
    """Test find_upgrade_opportunities finds Tier 1 → Tier 2 upgrades"""
    print("\n=== Testing find_upgrade_opportunities ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Analyze routine
    routine_analysis = analyze_customer_routine(customer, product_lookup)

    # C001 has tier_preference: 2, so should be eligible
    # Find upgrade opportunities
    opportunities = find_upgrade_opportunities(
        customer,
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    # Verify structure
    assert isinstance(opportunities, list), "Should return a list"

    # Verify each opportunity has required fields (if any found)
    for opp in opportunities:
        assert "product_id" in opp, "Each opportunity should have product_id"
        assert "recommendation_type" in opp, "Each opportunity should have recommendation_type"
        assert opp["recommendation_type"] == "upgrade", "Should be upgrade type"
        assert "upgrades_from" in opp, "Each opportunity should have upgrades_from"

    print("✅ find_upgrade_opportunities test passed!")
    print(f"   Opportunities found: {len(opportunities)}")
    print(f"   Note: Tier 2 customers may not have Tier 1 products, so this may be empty")

    if opportunities:
        print(f"   Sample: {opportunities[0]['product_name']}")
        print(f"   Upgrades from: {opportunities[0]['upgrades_from']}")

    return opportunities


def test_find_replenishment_opportunities():
    """Test find_replenishment_opportunities converts needs to opportunities"""
    print("\n=== Testing find_replenishment_opportunities ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Get replenishment needs (using routine_analysis utility)
    from agents.tier2_crosssell_upsell.utilities.routine_analysis import check_replenishment_needs

    replenishment_needs = check_replenishment_needs(customer, product_lookup)

    # Find replenishment opportunities
    opportunities = find_replenishment_opportunities(
        replenishment_needs,
        product_lookup
    )

    # Verify structure
    assert isinstance(opportunities, list), "Should return a list"

    # Verify each opportunity has required fields
    for opp in opportunities:
        assert "product_id" in opp, "Each opportunity should have product_id"
        assert "recommendation_type" in opp, "Each opportunity should have recommendation_type"
        assert opp["recommendation_type"] == "replenishment", "Should be replenishment type"
        assert "days_since_purchase" in opp, "Each opportunity should have days_since_purchase"
        assert "replenishment_due" in opp, "Each opportunity should have replenishment_due"

    print("✅ find_replenishment_opportunities test passed!")
    print(f"   Replenishment needs: {len(replenishment_needs)}")
    print(f"   Opportunities found: {len(opportunities)}")

    if opportunities:
        print(f"   Sample: {opportunities[0]['product_name']}")
        print(f"   Due: {opportunities[0]['replenishment_due']}, "
              f"Days: {opportunities[0]['days_since_purchase']}")

    return opportunities


def test_opportunity_detection_integration():
    """Test all opportunity detection utilities together"""
    print("\n=== Testing opportunity detection integration ===")

    # Load data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Analyze routine
    routine_analysis = analyze_customer_routine(customer, product_lookup)

    # Get replenishment needs
    from agents.tier2_crosssell_upsell.utilities.routine_analysis import check_replenishment_needs
    replenishment_needs = check_replenishment_needs(customer, product_lookup)

    # Find all opportunities
    cross_sell = find_cross_sell_opportunities(
        routine_analysis["customer_routine_steps"],
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    ingredient = find_ingredient_based_opportunities(
        customer,
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    upgrade = find_upgrade_opportunities(
        customer,
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    replenishment = find_replenishment_opportunities(
        replenishment_needs,
        product_lookup
    )

    print("✅ Integration test passed!")
    print(f"   Customer: {customer['name']} ({customer['customer_id']})")
    print(f"   Cross-sell opportunities: {len(cross_sell)}")
    print(f"   Ingredient-based opportunities: {len(ingredient)}")
    print(f"   Upgrade opportunities: {len(upgrade)}")
    print(f"   Replenishment opportunities: {len(replenishment)}")
    print(f"   Total opportunities: {len(cross_sell) + len(ingredient) + len(upgrade) + len(replenishment)}")

    return {
        "cross_sell": cross_sell,
        "ingredient": ingredient,
        "upgrade": upgrade,
        "replenishment": replenishment
    }


if __name__ == "__main__":
    """Run all tests"""
    print("=" * 60)
    print("Testing Tier 2 Opportunity Detection Utilities")
    print("=" * 60)

    # Test individual utilities
    test_find_cross_sell_opportunities()
    test_find_ingredient_based_opportunities()
    test_find_upgrade_opportunities()
    test_find_replenishment_opportunities()

    # Integration test
    test_opportunity_detection_integration()

    print("\n" + "=" * 60)
    print("✅ All opportunity detection utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_opportunity_detection
============================================================
Testing Tier 2 Opportunity Detection Utilities
============================================================

=== Testing find_cross_sell_opportunities ===
✅ find_cross_sell_opportunities test passed!
   Opportunities found: 2
   Sample: HydraBalance Prep Toner (step 2)
   Rationale: Customer missing routine step 2. Adjacent to existing steps.

=== Testing find_ingredient_based_opportunities ===
✅ find_ingredient_based_opportunities test passed!
   Opportunities found: 2
   Sample: HydraBalance Prep Toner
   Matched concerns: ['dryness', 'sensitivity']
   Match score: 1.0

=== Testing find_upgrade_opportunities ===
✅ find_upgrade_opportunities test passed!
   Opportunities found: 0
   Note: Tier 2 customers may not have Tier 1 products, so this may be empty

=== Testing find_replenishment_opportunities ===
✅ find_replenishment_opportunities test passed!
   Replenishment needs: 6
   Opportunities found: 2
   Sample: Clarifying Deep Cleanse Foam
   Due: False, Days: 23

=== Testing opportunity detection integration ===
✅ Integration test passed!
   Customer: Sarah Lee (C001)
   Cross-sell opportunities: 2
   Ingredient-based opportunities: 2
   Upgrade opportunities: 0
   Replenishment opportunities: 2
   Total opportunities: 6

============================================================
✅ All opportunity detection utility tests passed!


#Opportunity Detection Node

In [ ]:

def opportunity_detection_node(state: Tier2CrossSellUpsellState) -> Dict[str, Any]:
    """
    Opportunity Detection Node: Find all cross-sell and upsell opportunities.

    This node orchestrates opportunity detection using the opportunity detection utilities.
    It finds cross-sell, ingredient-based, upgrade, and replenishment opportunities.

    Args:
        state: Current state with customer_data, product_catalog, product_lookup,
               customer_products, customer_routine_steps, replenishment_needs

    Returns:
        Updated state with:
        - cross_sell_opportunities: Routine step-based opportunities
        - ingredient_opportunities: Ingredient/concern matching opportunities
        - upgrade_opportunities: Tier 1 → Tier 2 upgrade opportunities
        - replenishment_opportunities: Replenishment opportunities
        - bundle_opportunities: Multi-product bundles (empty for now, can be added later)
    """
    errors = state.get("errors", [])

    # Get required data from state
    customer_data = state.get("customer_data")
    product_catalog = state.get("product_catalog")
    product_lookup = state.get("product_lookup")
    customer_products = state.get("customer_products", [])
    customer_routine_steps = state.get("customer_routine_steps", [])
    replenishment_needs = state.get("replenishment_needs", [])

    if not customer_data:
        return {
            "errors": errors + ["opportunity_detection_node: customer_data is required"]
        }

    if not product_catalog:
        return {
            "errors": errors + ["opportunity_detection_node: product_catalog is required"]
        }

    if not product_lookup:
        return {
            "errors": errors + ["opportunity_detection_node: product_lookup is required"]
        }

    try:
        # Step 1: Find cross-sell opportunities (routine step-based)
        cross_sell_opportunities = find_cross_sell_opportunities(
            customer_routine_steps,
            customer_products,
            product_catalog,
            product_lookup
        )

        # Step 2: Find ingredient-based opportunities
        ingredient_opportunities = find_ingredient_based_opportunities(
            customer_data,
            customer_products,
            product_catalog,
            product_lookup
        )

        # Step 3: Find upgrade opportunities
        upgrade_opportunities = find_upgrade_opportunities(
            customer_data,
            customer_products,
            product_catalog,
            product_lookup
        )

        # Step 4: Find replenishment opportunities
        replenishment_opportunities = find_replenishment_opportunities(
            replenishment_needs,
            product_lookup
        )

        # Step 5: Bundle opportunities (empty for now, can be added later)
        # Bundle logic: if customer has 3+ routine gaps, suggest bundles
        bundle_opportunities = []
        # TODO: Implement bundle logic if needed

    except Exception as e:
        return {
            "errors": errors + [f"opportunity_detection_node: Error during opportunity detection: {str(e)}"]
        }

    return {
        "cross_sell_opportunities": cross_sell_opportunities,
        "ingredient_opportunities": ingredient_opportunities,
        "upgrade_opportunities": upgrade_opportunities,
        "replenishment_opportunities": replenishment_opportunities,
        "bundle_opportunities": bundle_opportunities,
        "errors": errors
    }


# Test Node

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_nodes
============================================================
Testing Tier 2 Cross-Sell & Upsell Orchestrator Nodes
============================================================

=== Testing goal_node ===
✅ goal_node test passed!
   Goal: Identify cross-sell and upsell opportunities for Tier 2 customer
   Focus areas: ['routine_completeness', 'ingredient_matching', 'upgrade_opportunities', 'replenishment_needs', 'routine_step_adjacency']

=== Testing goal_node with missing customer_id ===
✅ goal_node error handling test passed!
   Error: goal_node: customer_id is required

=== Testing planning_node ===
✅ planning_node test passed!
   Plan has 6 steps:
      Step 1: data_loading
      Step 2: routine_analysis
      Step 3: opportunity_detection
      Step 4: scoring
      Step 5: ranking
      Step 6: report_generation

=== Testing planning_node with missing goal ===
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

=== Testing goal_node → planning_node integration ===
✅ Integration test passed!
   Customer ID: C001
   Goal: Identify cross-sell and upsell opportunities for Tier 2 customer
   Plan steps: 6

=== Testing data_loading_node ===
✅ data_loading_node test passed!
   Customer: Sarah Lee (C001)
   Products in catalog: 20
   Lookup size: 20

=== Testing data_loading_node with missing customer_id ===
✅ data_loading_node error handling test passed!
   Error: data_loading_node: customer_id is required

=== Testing data_loading_node with customer not found ===
✅ data_loading_node (customer not found) test passed!
   Error: data_loading_node: Customer C999 not found

=== Testing goal_node → planning_node → data_loading_node integration ===
✅ Integration test passed!
   Customer ID: C001
   Customer: Sarah Lee
   Products in catalog: 20
   Plan steps: 6

=== Testing routine_analysis_node ===
✅ routine_analysis_node test passed!
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

=== Testing routine_analysis_node with missing data ===
✅ routine_analysis_node error handling test passed!
   Error: routine_analysis_node: customer_data is required

=== Testing full integration (goal → planning → data_loading → routine_analysis) ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

=== Testing opportunity_detection_node ===
✅ opportunity_detection_node test passed!
   Cross-sell opportunities: 2
   Ingredient opportunities: 2
   Upgrade opportunities: 0
   Replenishment opportunities: 2
   Bundle opportunities: 0

=== Testing opportunity_detection_node with missing data ===
✅ opportunity_detection_node error handling test passed!
   Error: opportunity_detection_node: customer_data is required

=== Testing full integration through opportunity_detection ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Cross-sell: 2
   Ingredient: 2
   Upgrade: 0
   Replenishment: 2
   Total opportunities: 6

============================================================
✅ All tests passed!
============================================================
